# RSS Feed Agent

1. Write up the tools necessary to:
   1. Search websites for new info
   2. Write an organized report to a file
2. Write up the agents and setup their communication
3. Test the pipeline

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [4]:
import requests
from bs4 import BeautifulSoup
import time


def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import requests
    from bs4 import BeautifulSoup
    import time
    
    url = "https://customsearch.googleapis.com/customsearch/v1"
    params = {
        "key": os.environ["GOOGLE_API_KEY"],
        "cx": os.environ["GOOGLE_CSE_ID"],
        "q": query,
        "num": num_results
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        raise Exception(f"Error in API request: {response.status_code}")
        
    results = response.json().get("items", [])
    
    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            return text[:max_chars]
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""
            
    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append({
            "title": item["title"],
            "link": item["link"], 
            "snippet": item["snippet"],
            "body": body
        })
        time.sleep(1)
        
    return enriched_results

In [6]:
output = google_search("hacker news AI")

In [7]:
for result in output:
    print(result)

{'title': 'How I Use "AI" | Hacker News', 'link': 'https://news.ycombinator.com/item?id=41150317', 'snippet': "As a device for speeding up learning, they're incredible. Best of all, they're strongest where I'm weakest: finding all the arbitrary details that are needed\xa0...", 'body': 'How I Use "AI" | Hacker News Hacker News new | past | comments | ask | show | jobs | submit login How I Use "AI" ( carlini.com ) 462 points by npalli 5 months ago | hide | past | favorite | 180\xa0comments tptacek 5 months ago | next [–] There\'s a running theme in here of programming problems LLMs solve where it\'s actually not that important that the LLM is perfectly correct. I\'ve been using GPT4 for the past couple months to comprehend Linux kernel code; it\'s spooky good at it. I\'m a C programmer,'}
{'title': 'What it feels like to work in AI right now | Hacker News', 'link': 'https://news.ycombinator.com/item?id=35469908', 'snippet': "Apr 6, 2023 ... Yes, that's called 'ensembling'. There is a lot

In [8]:
def write_report(summary_insights: str, report_file: str):
    """
    Write a summary of the insights to a file
    """
    with open(report_file, "w") as f:
        f.write(summary_insights)

write_report("AI is the future", "report.txt")

In [11]:
from autogen_core.tools import FunctionTool

search_tool = FunctionTool(
    func=google_search,
    name="search_tool",
    description="Search the web for information",
)

write_report_tool = FunctionTool(
    func=write_report,
    name="write_report_tool",
    description="Write a summary of the insights to a file",
)


# Let's build out Agents!

We'll have 3 agents:
1. To Search the web
2. The extract insights from web results
3. Write those insights to file

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken

In [24]:
search_agent = AssistantAgent(
    name="search_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o",
    ),
    system_message="You research the web for information and you communicate your findings to the insights_agent",
    tools=[search_tool],
)

model_client_structured = OpenAIChatCompletionClient(
    model="gpt-4o",
    response_format={"type": "json_object"},
)

insights_agent = AssistantAgent(
    name="insights_agent",
    model_client=model_client_structured,
    system_message="You 10 extract insights from the web results and you communicate your findings to the write_report_agent using a json structure.",
)

write_report_agent = AssistantAgent(
    name="write_report_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o",
    ),
    system_message="You write a report from the insights provided by the insights_agent using the write_report_tool. You always output\
        the word 'DONE' when you are finished.",
    tools=[write_report_tool],
)

In [25]:
cancellation_token = CancellationToken()

In [26]:
from autogen_agentchat.conditions import TextMentionTermination

In [27]:
from autogen_agentchat.teams import RoundRobinGroupChat

do_team = RoundRobinGroupChat(
    [search_agent, insights_agent, write_report_agent],
    termination_condition=TextMentionTermination(text="DONE"),
)

response = await do_team.run(task="Search hacker news for the latest open source LLMs and extact the best insights about the latest models into a file and call it rss-feed-do.md")

print(response)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Search hacker news for the latest open source LLMs and extact the best insights about the latest models into a file and call it rss-feed-do.md', type='TextMessage'), ToolCallRequestEvent(source='search_agent', models_usage=RequestUsage(prompt_tokens=135, completion_tokens=32), content=[FunctionCall(id='call_eWWLbueFAO9hYltsW6fmuFFL', arguments='{"query":"site:news.ycombinator.com open source LLMs","num_results":5}', name='search_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='search_agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': \'In the LLM space, "open source" is being used to mean ...\', \'link\': \'https://news.ycombinator.com/item?id=36815255\', \'snippet\': "Jul 21, 2023 ... LLaMA is not open weights. It\'s a closed, proprietary set of weights[0] that at best could be compared to source available software.", \'body\': \'In the LLM space, "open sourc